### ORI Final Analysis
### 04. Industry, Navigation, and Transportation

#### 10/24/2024
#### This script processes data and creates the natioanl security industry, navigation, and transportation submodel. 

In [1]:
#import system modules and set environment
import arcpy
import os
import pandas as pd
import numpy as np
from arcpy import env
from arcpy.sa import *
arcpy.env.overwriteOutput = True
arcpy.env.workspace = 'C:/Users/Eliza.Carter/Documents/Projects/California/ORI'

In [2]:
#define spatial reference (EPSG for NAD 83 UTM Zone 11N)
spatial_ref = arcpy.SpatialReference(26911)

#create a copy of the ORI area hex grid
#hex_grid = 'data/ORI_Area_of_Interest_grid.shp'
hex_grid = 'data/input_data/constrained.shp'
industry_submodel = 'data/input_data/industry_submodel'
arcpy.management.CopyFeatures(hex_grid, industry_submodel)

#create geopackage
in_gpkg_path = 'data/input_data/ORI_Industry.gpkg'
if not os.path.exists(in_gpkg_path):
    arcpy.management.CreateSQLiteDatabase(in_gpkg_path, "GEOPACKAGE")

In [3]:
# BOEM Active Lease Blocks (ori_in_01)

#define file
file_path = 'data/source_data/BOEM_Pacific_Leases.shp'

#create feature layer for the file
arcpy.management.MakeFeatureLayer(file_path, "active_leases")

#select file where it itersects the grid
arcpy.management.SelectLayerByLocation(
    in_layer= "active_leases",
    overlap_type="INTERSECT",
    select_features= industry_submodel,
    search_distance=None,
    selection_type="NEW_SELECTION",
    invert_spatial_relationship="NOT_INVERT"
)

#project to defined spatial reference
file_project = 'in_memory/file_projected'
arcpy.management.Project("active_leases", file_project, spatial_ref)                                            

#save to geopackage
output_name = 'Active_BOEM_Leases'
arcpy.conversion.FeatureClassToFeatureClass(file_project, in_gpkg_path, output_name)

#identify overlap between this layer and the hex grid; score overlapping cells using predetermined value
file_selection = arcpy.management.SelectLayerByLocation(industry_submodel,
                                                                     'INTERSECT',
                                                                     file_project)
arcpy.management.CalculateField(file_selection, 'ori_in_01', '0.5', field_type = "DOUBLE")

file_selection = arcpy.management.SelectLayerByLocation(industry_submodel, 
                                                        'INTERSECT', 
                                                        file_project, 
                                                        invert_spatial_relationship=True)
arcpy.management.CalculateField(file_selection, 'ori_in_01', '1', field_type="DOUBLE")

<Result 'industry_submodel_Layer2'>

In [4]:
# Combine and project all AIS transects that overlap the hex grid

#paths
gpkg_path = 'data/source_data/wc_ais_transect.gpkg'
initial_selected_path = 'data/source_data/ori_ais_transects_temp.shp'
final_projected_path = 'data/source_data/ori_ais_transects.shp'

#list gpkg layers
gpkg_description = arcpy.Describe(gpkg_path)
layer_names = [layer.name for layer in gpkg_description.children]

#select the first layer
first_layer_path = f"{gpkg_path}/{layer_names[0]}"
arcpy.management.MakeFeatureLayer(first_layer_path, "first_ais_transects")

#select features in the first layer that intersect the hex grid
arcpy.management.SelectLayerByLocation(
    in_layer="first_ais_transects",
    overlap_type="INTERSECT",
    select_features=industry_submodel,
    search_distance=None,
    selection_type="NEW_SELECTION"
)

#copy only intersecting features to start the combined AIS layer
arcpy.management.CopyFeatures("first_ais_transects", initial_selected_path)
arcpy.management.Delete("first_ais_transects")

#loop through remaining layers and append only selected features
for layer_name in layer_names[1:]:
    layer_path = f"{gpkg_path}/{layer_name}"
    
    #create a feature layer
    arcpy.management.MakeFeatureLayer(layer_path, "ais_transects")
    
    #select features that intersect the hex grid
    arcpy.management.SelectLayerByLocation(
        in_layer="ais_transects",
        overlap_type="INTERSECT",
        select_features=industry_submodel,
        search_distance=None,
        selection_type="NEW_SELECTION"
    )
    
    #append selected features
    arcpy.management.Append("ais_transects", initial_selected_path, "NO_TEST")
    
    #clean up
    arcpy.management.Delete("ais_transects")

# project combined AIS transects
arcpy.management.Project(initial_selected_path, final_projected_path, spatial_ref)

#clean up
arcpy.management.Delete(initial_selected_path)


<Result 'true'>

###  Run AIS Processing Script here

In [5]:
# Join AIS to constrained hex grid

#define file
file_path = 'data/input_data/ORI_Industry.gpkg'
layer_name = 'main.ORI_AIS_Grid_Intersect'
field_list = ["Cargo", "Tug_Tow", "Fishing", "Passenger", "Pleasure", "Other"]

#join to industry submodel
arcpy.management.JoinField(industry_submodel, "GRID_ID", f"{file_path}/{layer_name}", "GRID_ID", field_list)


<Result 'C:/Users/Eliza.Carter/Documents/Projects/California/ORI\\data/input_data/industry_submodel'>

In [6]:
# Export Industry submodel as a csv

#create new row number field
new_field_name = "RowNum"
arcpy.AddField_management(industry_submodel, new_field_name, "LONG")

#fill in field with row numbers
with arcpy.da.UpdateCursor(industry_submodel, new_field_name) as cursor:
    row_number = 1
    for row in cursor:
        row[0] = row_number
        cursor.updateRow(row)
        row_number += 1 

#export the table
ais_path = "data/input_data/ORI_AIS.csv"
arcpy.conversion.TableToTable(industry_submodel, os.path.dirname(ais_path), os.path.basename(ais_path))

<Result 'C:/Users/Eliza.Carter/Documents/Projects/California/ORI\\data/input_data\\ORI_AIS.csv'>

In [7]:
#Alter AIS fields with z-membership scores

#perform z-membership on each field and save 
input_table = "C:/Users/Eliza.Carter/Documents/Projects/California/ORI/data/input_data/ORI_AIS.csv"
output_table = "C:/Users/Eliza.Carter/Documents/Projects/California/ORI/data/input_data/ORI_AIS_z.csv"

#User Needs to input list of field names to be rescaled
z_columns = ["Cargo", "Tug_Tow", "Fishing", "Passenger", "Pleasure", "Other"]
final_name = ["ori_in_02", "ori_in_03", "ori_in_04", "ori_in_05", "ori_in_06", "ori_in_07"]

#Custom Z membership function rescales data between 0 and 1
def cust_zmf(x, a, b):
    assert a <= b, 'a <= b is required.'
    na = np.empty(len(x))
    na[:] = np.isnan(x)
    na[na == 1] = np.nan
    na[na == 0] = 1
    zy = np.ones(len(x))
    idx = np.logical_and(a <= x, x < (a + b) / 2.)
    zy[idx] = (1 - 2 * ((x[idx] - a) / (b - a)) ** 2.)
    idx = np.logical_and((a + b) / 2. <= x, x <= b)
    zy[idx] = (2. * ((x[idx] - b) / (b - a)) ** 2.)
    idx = x >= b
    zy[idx] = 0
    z = np.multiply(na, zy)
    return z

df = pd.read_csv(input_table)
OID = df["RowNum"]
df_fin = pd.DataFrame(OID, columns=["RowNum"])

for fz in z_columns:
    ff = df[fz]
    ceiling = np.nanmin(ff)
    foot = np.nanmax(ff) + np.nanmax(ff) / 10000 #This is performed to ensure no 0 values in output
    f = cust_zmf(ff, ceiling, foot)
    f_col = pd.DataFrame(f, columns=[fz])
    df_fin = pd.concat([df_fin,f_col], axis=1)

df_final_all = df_fin[z_columns]
df_final_all.columns = final_name[:len(df_final_all.columns)] #rename columns
df_final_all["joiner"] = df[["RowNum"]]
df_final_all.to_csv(output_table, index=False)


In [17]:
# Join z-membership fields to the industry submodel
csv_path = "data/input_data/ORI_AIS_z.csv"

arcpy.management.JoinField(industry_submodel, "RowNum", csv_path, "joiner", fields="joiner;ori_in_02;ori_in_03;ori_in_04;ori_in_05;ori_in_06;ori_in_07")


<Result 'C:/Users/Eliza.Carter/Documents/Projects/California/ORI\\data/input_data/industry_submodel.shp'>

In [18]:
# Industry Submodel

#call shapefile
industry_submodel = 'data/input_data/industry_submodel.shp'

#create a new field for national security
in_field = "in_final"

existing_fields = [f.name for f in arcpy.ListFields(industry_submodel)]

if in_field not in existing_fields:
    arcpy.management.AddField(industry_submodel, in_field, 'DOUBLE')
    
#expression to get final constraints
final_expression = '(!ori_in_01! * !ori_in_02! * !ori_in_03! * !ori_in_04! * !ori_in_05! * !ori_in_06! * !ori_in_07!) ** (1/7)'

# Calculate the geometric mean and store the result in the 'final_in' attribute
arcpy.CalculateField_management(industry_submodel, in_field, final_expression, 'PYTHON3')

<Result 'C:/Users/Eliza.Carter/Documents/Projects/California/ORI\\data/input_data/industry_submodel.shp'>